In [1]:
from mnemonic import Mnemonic
from fastecdsa import keys, curve
import binascii
import hashlib
import hmac



In [2]:
"""here i create a class to store data in a tree like structure"""

class TreeNode():
    
    def __init__(self,parent):
        self.parent=parent
        self.pvt_key=None
        self.pub_key=None
        self.chain_code =None
        self.children = []
    

In [3]:
"""Here I define the function needed to generate the seed which is going to be used later """

def generate_seed(language,passphrase,strenght):
   mnemo = Mnemonic(language)
   words = mnemo.generate(strength=strenght)
   seed = mnemo.to_seed(words, passphrase=passphrase)
   print("Entropy=" + str( binascii.hexlify(mnemo.to_entropy(words)))) #prints the entropy in hex
   print("Words= " + words)   
   print("Seed = "+str(binascii.hexlify(seed))) #prints the seed in hex 
   return seed
   


In [5]:
seed=generate_seed("english","test",128)

Entropy=b'cc00597a57b995cb65fd0b6631a3971a'
Words= slot actress future quarter off toss obey pause great minimum total cupboard
Seed = b'614f436abdb17cb30eaf9e195711ef9dee3599e31d8867979a4b96fd995420db6c4d39d4ea45fb6ee9b5affd2bff2674902c0aa9f215b97457166de07943d428'


In [7]:
"""In this section I derive from the seed the master private key and the master chain code, the private key is the one used to have control over your bank account represented by the public key. The bitcoin address is generated/corresponds from the public key"""
"""def get_master_info(seed):
    to_return={}
    master= hmac.new(seed,None,hashlib.sha512).hexdigest()
    master_pvt_key=master[:64]
    master_chain_code=master[64:]
    pvt=int(master_pvt_key, 16)
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1)
    to_return["master_pvt_key"] = master_pvt_key
    to_return["master_pub_key"] = master_pub_key
    to_return["master_chain_code"] = master_chain_code
    return to_return"""

def get_master_info(seed):
    master=TreeNode(None)
    master_hmac= hmac.new(seed,None,hashlib.sha512).hexdigest()
    master_pvt_key=master_hmac[:64]
    master.pvt_key=master_pvt_key
    master_chain_code=master_hmac[64:]
    master.chain_code=master_chain_code
    pvt=int(master_pvt_key, 16)
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1)
    pub_key= hex(master_pub_key.y)[2:4]+hex(master_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
    master.pub_key=pub_key
    return master


In [8]:
master_info=get_master_info(seed)
master_info.pub_key


'5f23703d704c907fde4e4fad47bc966e04d466d3d477884b50d987b1a44af167d7'

In [9]:
"""In this section I define a function to derive child addresses using hardened derivation"""

def create_hardened_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pvt_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)
            

In [10]:
"""in this section I create a children from the public key of the master"""

def create_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pub_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)

In [11]:
create_children(master_info,"00000000",2)
create_hardened_children(master_info,"80000000",1)

for i in master_info.children:
        print("Private Key " + i.pvt_key)
        print("Public Key " + i.pub_key)
        print("Chain Code " + i.chain_code)
        print("-------------------------------------------------------------------------")
        

Private Key 1e4f2fbf7fefa5c5c88816e6d23f811bcd382b70e16f73571fc2951d536dd1a4
Public Key 608836f152bc6ab0867757cc7594157d2188bab3beea506d4264d7a2424c51bf09
Chain Code 50c8147bbe16d175008ff1aa7827e72d349877c28b193fe14be095c0ec0f710f
-------------------------------------------------------------------------
Private Key 16bbae1d2e4ff333d334b502de78fa108921814fbc920a40cd8e7933ea93c484
Public Key 7abcf5f552c75dbcc7e64132cb39fee19a6b4b734a7e26522fd78c62c6248494ee
Chain Code 912d78769ee55b93cb279c16f768aedb4a1df6fb8318d0e68f7bb026cc55047f
-------------------------------------------------------------------------
Private Key b60d828ef9102e0286db1b979e787149d068c18cc25fd581170d20c61d5ebf3e
Public Key 921e635704a31cfe9cd4e047e47e0116b753049ff6b27185df698cbb5d70ceb05d
Chain Code 3d3a6ea0bd1ccba4e72c3b246e2baf4a7f6775ac8b3408a51b0d1751af03f66c
-------------------------------------------------------------------------


In [12]:
"""Lets try to create some grandkids"""

for i in master_info.children:
    create_hardened_children(i,"80000000",1)
    create_children(i,"00000000",2)
    for j in i.children:
        print("Private Key " + j.pvt_key)
        print("Public Key " + j.pub_key)
        print("Chain Code " + j.chain_code)
        print("-------------------------------------------------------------------------")
        




Private Key e98789c8fb900856a05763160be51a8774716afffee8d1794399a17466dcdb3b
Public Key d7f9d2e639746011c9d73233f8d5fcf374aba2c043d57d66f71ed885d987c3166f
Chain Code 4b9ad712787865c8bfad797596cff9f92bb40bbc2163925f29e27e428d6d78c8
-------------------------------------------------------------------------
Private Key 050a1428351b3f668249df7d905a24d3960f146e8a7ff993f3c35f1e48030fab
Public Key 49f40d92f6d858d9aa7eb2e4b48778593c9bc337d8272cb2ee6bb27834f02b537c
Chain Code a66aa3f7ebcae49284ed7cc111908473a6464e3980fe94e88a1ebb310ee87e9b
-------------------------------------------------------------------------
Private Key 6bed8f1e3f82be147674233a83645d9be26d534a653280598667529fb194fdb6
Public Key c9ab4d665008013c013d51594787d669197a35f9fd1fe21b424f5aafdce9b15058
Chain Code 0ce7996e6dafbb93173030f8adcb2ddbf9a889caeed1b847e22532414d2b1ac7
-------------------------------------------------------------------------
Private Key 87ed108efd4aa36c9667a8da8d06b7020bd28de5c67429aa930b323953747e7b
Public K